## ปฏิบัติการที่ 3.1 นักศึกษาสามารถติดตั้ง Spark ร่วมกับ Python ได้

### 1. ทำการติดตั้งเครื่องมือที่จำเป็นต้องใช้

In [2]:
!apt-get update                                                                                                          # อัพเดท Package ทั้งหมดใน Colab VM
!apt-get install openjdk-8-jdk-headless -qq > /dev/null                                                   # ติดตั้ง Java Development Kit (จำเป็นสำหรับการติดตั้ง Spark)
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz  # ติดตั้ง Spark 3.2.1
!tar xzvf spark-3.2.1-bin-hadoop2.7.tgz                                                                         # Unzip ไฟล์ Spark 3.2.1
!pip install findspark                                                                                                    # ติดตั้ง Package Python สำหรับเชื่อมต่อกับ Spark 

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://archive.ubuntu.co

In [3]:
# Set enviroment variable ให้ Python รู้จัก Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [4]:
# ติดตั้ง PySpark ลงใน Python
!pip install pyspark==3.2.1

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 66.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=3b7a1f03f16eeb630383d9dd3607d3e628b80a12f43660480aee13c2c0f771ba
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


### 2. ทำการสร้าง Session สำหรับการทำงานของ Spark บน Colab

In [5]:
# ตรวจสอบว่า Server ของ Google Colab มีกี่ Core
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [6]:
# สร้าง Spark Session เพิ้อใช้งาน Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [7]:
# ดูเวอร์ชั่น Python
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=13, releaselevel='final', serial=0)

In [8]:
# ดูเวอร์ชั่น Spark
spark.version

'3.2.1'

## ปฏิบัติการที่ 3.2 (5 คะแนน) นักศึกษาสามารถสร้าง Data Profile ได้

### 3. อ่านไฟล์ CSV ด้วย Spark 


- ให้นำไฟล์ CSV เข้า Colab โดยใช้คำสั่ง spark.read.csv

สามารถอ่านเพิ่มเติมได้ [ที่นี่](https://spark.apache.org/docs/latest/sql-data-sources-csv.html)

**Note**

* Header = True หมายความว่า บรรทัดแรกในไฟล์ CSV เป็น Header
* Inferschema = True หมายความว่า Spark จะกำหนด Data type 
ของแต่ละ column ให้ (ถ้ากำหนดเป็น False คือการกำหนดให้ทุก Column มี Data Type เป็น String ทั้งหมด)
* sep คือ Delimit Separator

In [10]:
car_dt = spark.read.csv('/content/cars.csv', header = True, inferSchema = True )
car_dt.show(5)

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|       130|  3504|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|       165|  3693|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|       150|  3436|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|       150|  3433|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|       140|  3449|        10.5|   70|    US|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
only showing top 5 rows



- ให้อ่าน CSV จาก URL 

In [ ]:
url = "https://perso.telecom-paristech.fr/eagan/class/igr204/data/cars.csv"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
car_dt = spark.read.csv("file://"+SparkFiles.get("cars.csv"), header=True, inferSchema= True, sep =';',)

In [ ]:
car_dt.select("Car","MPG").show()

### 4. ดู Schema ของ car_dt โดยใช้คำสั่ง printSchema

In [ ]:
car_dt.printSchema()

### 5. แสดงจำนวน Row และ Column รวมทั้งแสดงรายละเอียดของ Column โดยใช้คำสั่ง count() และ columns


In [ ]:
#จำนวน Column
print("The number of rows: ", car_dt.count())
print("The number of column: ", len(car_dt.columns), " and list of columns are ", car_dt.columns)

### 6. แสดงข้อมูลสรุปสถิติของแต่ละ Column โดยใช้คำสั่ง describe() หรือ summary()

In [ ]:
car_dt.describe().show()

In [ ]:
car_dt.summary().show()

### 7.  ให้นักศึกษาที่การอ่าน CSV จาก url เข้ามาเก็บไว้ที่ตัวแปร country_dt 

ให้แสดงผลเชิงสรุปโดยใช้คำสั่ง summary() (เลือกเฉพาะ count มาแสดง) โดยเลือกเฉพาะ Column (ใช้คำสั่ง select()) ที่ชื่อว่า 
•	Country
•	Birth rate(births/1000 population)
•	Exports


In [ ]:
url = "https://perso.telecom-paristech.fr/eagan/class/igr204/data/factbook.csv"
from pyspark import SparkFiles
spark.sparkContext.addFile(url)
country_dt = spark.read.csv("file://"+SparkFiles.get("factbook.csv"), header=True, inferSchema= True, sep =';',)

In [ ]:
country_dt = country_dt.where(country_dt.Country != "String")
country_dt.show(10)

In [ ]:
print("The number of rows: ", country_dt.count())
country_dt.select("Country","Birth rate(births/1000 population)", "Exports").summary("count").show()

### 8. (1 คะแนน) ให้แสดงแถวข้อมูลที่มี Missing Value หรือ Null ที่เกิดใน Column ชื่อ Exports โดยใช้คำสั่ง where หรือ filter แสดงรายละเอียดของ Column ชื่อ Country และ Exports และแสดงเฉพาะ 10 แถวแรก 

การใช้ Filter และ Where อ่านเพิ่มเติมได้ [ที่นี่](https://sparkbyexamples.com/spark/spark-filter-rows-with-null-values/) หรือ [ที่นี่](https://sparkbyexamples.com/pyspark/pyspark-where-filter/)

In [ ]:
country_dt.printSchema()

In [ ]:
country_dt.where(country_dt.Exports.isNull()).select("Country","Exports").show(10)

In [ ]:
country_dt.filter("Exports is NULL").select("Country", "Exports").show(10)

## ปฏิบัติการที่ 3.3 (5 คะแนน) นักศึกษาสามารถใช้งาน EDA ได้

### 9. ให้ทำการนำเข้าไฟล์ CSV ชื่อ cereal.csv เข้า Colab และดู Schema

In [7]:
cereal_dt = spark.read.csv('/content/cereal.csv', header = True, inferSchema = True )
cereal_dt.show(5)

+--------------------+---+----+-----------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+-------------------+
|                name|mfr|type|   calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|    record_datetime|
+--------------------+---+----+-----------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+-------------------+
|           100% Bran|  N|   C|70.44442454|   null|   1|   130| 10.0|  5.0|     6|   280|    null|    3|   1.0|0.33|68.402973|2022-06-11 14:04:45|
|   100% Natural Bran|  Q|   C|120.9993938|      3|   5|    15|  2.0|  8.0|  null|   135|       0|    3|   1.0| 1.0|33.983679|2021-09-20 14:35:58|
|            All-Bran|  K|   C|70.56453146|      4|null|  null|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|2022-04-02 03:43:03|
|All-Bran with Ext...|  K|   C|50.62726767|      4|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93

In [ ]:
cereal_dt.printSchema()

### 10. (1 คะแนน) ทำการกรองข้อมูลโดยใช้ filter โดยมีเงื่อนไขคือ เลือกแถวที่ fat และ sodium เป็นค่าว่าง (สัญลักษณ์ & คือ “และ” )
ศึกษาเพิ่มได้ [ที่นี่](https://sparkbyexamples.com/pyspark/pyspark-filter-rows-with-null-values/)

In [ ]:
cereal_dt.filter(cereal_dt.fat.isNull() & cereal_dt.sodium.isNull()).show()

### 11. (1 คะแนน) จงหาว่าชื่อ (name) ของ cereal ใดบ้างที่มีปริมาณ sugars อยู่ระหว่าง 7-10 (ไม่รวม 7 และ 10) 

•	คำสั่งเขียนอย่างไร ถ้าใช้ filter

In [ ]:
cereal_dt.filter(cereal_dt.sugars > 7).filter(cereal_dt.sugars < 10).select("name", "sugars").show()

•	คำสั่งเขียนอย่างไร ถ้าใช้ where

In [ ]:
cereal_dt.where("sugars > 7 AND sugars < 10").select("name", "sugars").show()

### 12. (1 คะแนน) จงหาจำนวนข้อมูล โดยมีเงื่อนไขคือ mfr มีค่า P หรือ Q ให้ใช้คำสั่ง count ในการนับ

ศึกษาเพิ่มเติมได้ [ที่นี่](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.DataFrame.count.html)

In [ ]:
numpq = cereal_dt.where("mfr in ('P','Q')").count()
print("The number of rows containing P or Q in mfr is ", numpq)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
cereal_pd = cereal_dt.toPandas()
cereal_pd.set_index('name')
cereal_pd.head()

### 15. (1 คะแนน) จงสร้าง Bar chart จากการใช้ matplotlib โดยเลือกเฉพาะคอลัม protein และ fat และเลือกเฉพาะ name = ['Clusters', 'Froot Loops', 'Grape-Nuts'] มาแสดง

ศึกษาเพิ่มเติมได้ [ที่นี่](https://www.shanelynn.ie/bar-plots-in-python-using-pandas-dataframes/)


In [ ]:
select_cereal = ['Clusters', 'Froot Loops', 'Grape-Nuts']

select_name = list(cereal_pd.loc[cereal_pd['name'].isin(select_cereal)].iloc[:, 0])
select_row_protein = list(cereal_pd.loc[cereal_pd['name'].isin(select_cereal)].iloc[:, 4])
select_row_fat = list(cereal_pd.loc[cereal_pd['name'].isin(select_cereal)].iloc[:, 5])


plotdata = pd.DataFrame({
    "protein":select_row_protein,
    "fat":select_row_fat
    }, 
    index=select_name
)

plotdata.plot(kind="bar")
plt.title("Cereal's fat and protein")
plt.xlabel("Cereal name")
plt.ylabel("nutrients (mg)")



### 16. (1 คะแนน) จงสร้าง scatter plot โดยเลือกคอลัม rating เป็นแกน x และ calories เป็นแกน y 

In [ ]:
rating = list(cereal_pd.iloc[:, 15])
calories = list(cereal_pd.iloc[:, 3])

plotdata = pd.DataFrame({
    "rating":rating,
    "calories":calories
    }
)

plotdata.plot(kind='scatter', x='calories', y='rating', c="red")
plt.title("Cereal's calories")
plt.xlabel("Rating")
plt.ylabel("Calories")

17.(1 คะแนน) จาก cereal_dt ให้ทำการแปลง Data type จากคอลัม record_datetime ซึ่งมี Data Type เป็น String ให้เป็น Data Type ที่ถูกต้อง (แปลงเป็น timestamp)

ศึกษาเพิ่มเติมได้ [ที่นี่](https://sparkbyexamples.com/spark/spark-convert-string-to-timestamp-format/)

In [ ]:
cereal_dt.printSchema()

In [8]:
from pyspark.sql import functions as f

cereal_clean_dt = cereal_dt.withColumn("record_datetime",  f.to_timestamp(cereal_dt.record_datetime, 'yyyy-MM-dd HH:mm:ss'))
cereal_clean_dt.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: double (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)
 |-- record_datetime: timestamp (nullable = true)



### 18.(1 คะแนน) จาก cereal_dt ให้ทำการแปลง Data type จากคอลัม calories ซึ่งมี Data Type เป็น double ให้เป็น int ด้วยคำสั่ง withColumn และ cast จากนั้นทำการแสดง Schema

In [9]:
cereal_clean_dt = cereal_clean_dt.withColumn("calories",  cereal_clean_dt.calories.cast("Integer"))
cereal_clean_dt.printSchema()

root
 |-- name: string (nullable = true)
 |-- mfr: string (nullable = true)
 |-- type: string (nullable = true)
 |-- calories: integer (nullable = true)
 |-- protein: integer (nullable = true)
 |-- fat: integer (nullable = true)
 |-- sodium: integer (nullable = true)
 |-- fiber: double (nullable = true)
 |-- carbo: double (nullable = true)
 |-- sugars: integer (nullable = true)
 |-- potass: integer (nullable = true)
 |-- vitamins: integer (nullable = true)
 |-- shelf: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- cups: double (nullable = true)
 |-- rating: double (nullable = true)
 |-- record_datetime: timestamp (nullable = true)



### 19. (1 คะแนน) ให้ทำการแทนที่ค่าที่สะกดผิดในคอลัม name จากชื่อ Just Right Fruit & Nut ให้เป็น Just Right Fruit and Nut (DataFrame ใหม่ที่ได้ชื่อ cereal_clean_dt) ให้ใช้ Function ของ Spark ชื่อ withColumn, when และ otherwise จากนั้นให้ทำการแสดงผลลัพธ์

ศึกษาเพิ่มเติมได้ [ที่นี่](https://sparkbyexamples.com/pyspark/pyspark-update-a-column-with-value/)

In [14]:
from pyspark.sql.functions import when

cereal_clean_dt = cereal_clean_dt.withColumn("name", when(cereal_clean_dt['name'] == 'Just Right Fruit & Nut', 'Just Right Fruit and Nut').otherwise(cereal_clean_dt['name']))


In [18]:
cereal_clean_dt.where('name = "Just Right Fruit and Nut"').select('name').show(1,False)

+------------------------+
|name                    |
+------------------------+
|Just Right Fruit and Nut|
+------------------------+



### 20. (1 คะแนน) จงใช้คำสั่ง filter และ rlike ในการหาค่าในคอลัม name โดยให้หาเฉพาะค่าที่ประกอบด้วยตัวอักษระ (A-Z และ a-z) อักขระประเภท whitespace (ช่องว่าง ใช้สัญลักษณ์ \\s) และเครื่องหมาย – เท่านั้น และแสดงผลออกมา

In [10]:
cereal_correct = cereal_clean_dt.filter(~cereal_clean_dt["name"].rlike("^[A-Za-z\s-]+$"))
#cereal_incorrect = cereal_clean_dt.subtract(cereal_correct)

print("The number of incorrect data:", cereal_correct.count())
cereal_correct.select('name').show(100,False)

The number of incorrect data: 12
+----------------------+
|name                  |
+----------------------+
|100% Bran             |
|100% Natural Bran     |
|Basic 4               |
|Cap'n'Crunch          |
|Cracklin' Oat Bran    |
|Cream of Wheat (Quick)|
|Crispy Wheat & Raisins|
|Just Right Fruit & Nut|
|Nut&Honey Crunch      |
|Post Nat. Raisin Bran |
|Product 19            |
|Shredded Wheat 'n'Bran|
+----------------------+



In [11]:
cereal_correct = cereal_clean_dt.filter(cereal_clean_dt["name"].rlike("^[A-Za-z\s-]+$"))
cereal_incorrect = cereal_clean_dt.subtract(cereal_correct)

print("The number of incorrect data:", cereal_incorrect.count())
cereal_incorrect.select('name').show(100,False)

The number of incorrect data: 12
+----------------------+
|name                  |
+----------------------+
|100% Natural Bran     |
|Product 19            |
|100% Bran             |
|Basic 4               |
|Post Nat. Raisin Bran |
|Shredded Wheat 'n'Bran|
|Cream of Wheat (Quick)|
|Cap'n'Crunch          |
|Just Right Fruit & Nut|
|Nut&Honey Crunch      |
|Cracklin' Oat Bran    |
|Crispy Wheat & Raisins|
+----------------------+



### 21. จงเขียนคำสั่งต่อไปนี้ เพื่อดูว่าแต่ละคอลัมมีจำนวนข้อมูลที่เป็น NULL เท่าไร

In [19]:
from pyspark.sql.functions import col, sum, when, count

cereal_null_dt = cereal_clean_dt.select([count(when(col(c).isNull(), c)).alias(c) for c in cereal_clean_dt.columns])
cereal_null_dt.show()

+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+
|name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|rating|record_datetime|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+
|   0|  2|   3|       1|      2|  2|     4|    2|    2|     3|     1|       1|    2|     2|   0|     1|              0|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+



### 22. (1 คะแนน) จงเขียนคำสั่งให้ข้อมูลใน cereal_clean_dt ลบแถวที่มีค่า NULL ในคอลัม vitamins โดยใช้คำสั่ง na.drop ใน spark จากนั้นให้แสดงจำนวนค่า NULL ในแต่ละคอลัม (จำนวนค่า NULL ใน vitamins จะเป็น 0)

ศึกษาเพิ่มเติมได้ [ที่นี่](https://sparkbyexamples.com/spark/spark-dataframe-drop-rows-with-null-values/)

In [26]:
cereal_rnull_dt = cereal_clean_dt.na.drop(subset=["vitamins"])
cereal_rnull_dt.show()

+--------------------+---+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+-------------------+
|                name|mfr|type|calories|protein| fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|    record_datetime|
+--------------------+---+----+--------+-------+----+------+-----+-----+------+------+--------+-----+------+----+---------+-------------------+
|   100% Natural Bran|  Q|   C|     120|      3|   5|    15|  2.0|  8.0|  null|   135|       0|    3|   1.0| 1.0|33.983679|2021-09-20 14:35:58|
|            All-Bran|  K|   C|      70|      4|null|  null|  9.0|  7.0|     5|   320|      25|    3|   1.0|0.33|59.425505|2022-04-02 03:43:03|
|All-Bran with Ext...|  K|   C|      50|      4|   0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|2021-11-20 03:42:19|
|      Almond Delight|  R|   C|     110|      2|   2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|2019-12-26 03

In [27]:
cereal_rnull_dt = cereal_rnull_dt.select([count(when(col(c).isNull(), c)).alias(c) for c in cereal_clean_dt.columns])
cereal_rnull_dt.show()

+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+
|name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|rating|record_datetime|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+
|   0|  2|   3|       1|      1|  2|     4|    2|    2|     3|     1|       0|    2|     2|   0|     1|              0|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+



### 23. จงเขียนคำสั่งให้ข้อมูลใน cereal_clean_dt ลบแถวที่มีค่า NULL ในทุกคอลัม และบันทึกไว้ที่ cereal_final_dt จากนั้นให้ใช้คำสั่งข้อ 21 แสดงจำนวนค่า NULL ในแต่ละคอลัม

In [41]:
cereal_final_dt = cereal_clean_dt.na.drop('any')
cereal_final_dt.show()

+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+-------------------+
|                name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|   rating|    record_datetime|
+--------------------+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+---------+-------------------+
|All-Bran with Ext...|  K|   C|      50|      4|  0|   140| 14.0|  8.0|     0|   330|      25|    3|   1.0| 0.5|93.704912|2021-11-20 03:42:19|
|      Almond Delight|  R|   C|     110|      2|  2|   200|  1.0| 14.0|     8|    -1|      25|    3|   1.0|0.75|34.384843|2019-12-26 03:44:02|
|         Apple Jacks|  K|   C|     110|      2|  0|   125|  1.0| 11.0|    14|    30|      25|    2|   1.0| 1.0|33.174094|2021-06-29 14:11:51|
|         Bran Flakes|  P|   C|      90|      3|  0|   210|  5.0| 13.0|     5|   190|      25|    3|   1.0|0.67|53.313813|2022-04-04 17:00:12|

In [38]:
cereal_rnull_dt.select([count(when(col(c).isNull(), c)).alias(c) for c in cereal_clean_dt.columns]).show()


+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+
|name|mfr|type|calories|protein|fat|sodium|fiber|carbo|sugars|potass|vitamins|shelf|weight|cups|rating|record_datetime|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+
|   0|  0|   0|       0|      0|  0|     0|    0|    0|     0|     0|       0|    0|     0|   0|     0|              0|
+----+---+----+--------+-------+---+------+-----+-----+------+------+--------+-----+------+----+------+---------------+



### 24. บันทึก cereal_final_dt ลงไฟล์ final_cereal.csv โดยใช้คำสั่ง

In [43]:
cereal_final_dt.coalesce(1).write.csv('final_cereal.csv', header = True)